Este dataset trata-se de dados de companhias aéreas dos EUA que contêm comentários de passageiros com base no serviço prestado pelas companhias aéreas no Twitter.
O link pode ser acessado em: https://www.kaggle.com/datasets/welkin10/airline-sentiment


# Dicionário de Dados - 

- **tweet_id**: Identificador único para cada tweet.
- **airline_sentiment**: Sentimento do tweet em relação à companhia aérea, podendo ser positivo, negativo ou neutro.
- **airline_sentiment_confidence**: Grau de confiança no sentimento atribuído ao tweet, variando de 0 a 1, onde 1 indica alta confiança.
- **negativereason**: Razão fornecida para o tweet ser classificado como negativo.
- **negativereason_confidence**: Grau de confiança na razão fornecida para o tweet ser classificado como negativo, com valores variando de 0 a .
- **airline**: Nome da companhia aérea a qual o tweet se refere.
- **airline_sentiment_gold**: Sentimento atribuído ao tweet por um avaliador humano de alta confiança (padrão ouro).
- **name**: Nome do usuário que postou o tweet.
- **negativereason_gold**: Razão negativa atribuída ao tweet por um avaliador humano de alta confiança (padrão ouro).
- **retweet_count**: Número de vezes que o tweet foi retweetado.
- **text**: Texto do tweet.
- **tweet_coord**: Coordenadas geográficas de onde o tweet foi postado.
- **tweet_created**: Data e hora de criação do tweet.
- **tweet_location**: Localização mencionada no perfil do usuário no momento da postagem do tweet.
- **user_timezone**: Fuso horário do usuário que postou o tweet.


**1 - Selecionar um conjunto de dados de análise de sentimentos binários disponível em qualquer base de dados online, excluindo aqueles abordados em aula e no TP2. Implementar as técnicas de TF-IDF, Bag-of-Words e Bag-of-nGrams para análise dos dados. Ao concluir, apresentar uma análise dos resultados obtidos. Caso o conjunto de dados apresente desbalanceamento, solicita-se a utilização de métricas adicionais além da acurácia para avaliar o desempenho dos modelos.**

In [1]:
# Importações necessárias
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv("tweets.csv")
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [4]:
#selecionado as colunas relevantes
df = df[['text', 'airline_sentiment']]

In [5]:
df= df.drop_duplicates()

In [6]:
print(df['airline_sentiment'].value_counts())

negative    9087
neutral     3067
positive    2298
Name: airline_sentiment, dtype: int64


In [7]:
#Removando registros 'neutral' de 'airline_sentiment'
df = df[df['airline_sentiment'] != 'neutral']

In [8]:
print(df['airline_sentiment'].value_counts())

negative    9087
positive    2298
Name: airline_sentiment, dtype: int64


Obs: Com base nos resultados acima, observamos que (após a filtragem) possuímos um conjunto desbalanceado onde 9087 menções enquadram-se como **negativo** e 2298 como **positivo**.

In [9]:
#Verificando valores NaN
df.isnull().sum()

text                 0
airline_sentiment    0
dtype: int64

In [10]:
#Convertendo a coluna airline sentment em binário
df['airline_sentiment'] = df['airline_sentiment'].map({'negative': 0, 'positive': 1})

In [11]:
print(df['airline_sentiment'].value_counts())

0    9087
1    2298
Name: airline_sentiment, dtype: int64


In [12]:

# Iniciando o lematizador e stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Função para limpeza e lematização do texto
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+|\@\w+|\#', '', text) # Removendo URLs, menções e hashtags com regex
    text = re.sub(r'\W', ' ', text).lower() # Removendo caracteres não alfanuméricos e convertendo para minúsculas
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]) # Lematizando e removendo stopwords
    return text

#Aplicando a função
df['clean_text'] = df['text'].apply(clean_text)

In [13]:
df

,text,airline_sentiment,clean_text
1,@VirginAmerica plus you've added commercials t...,1,plus added commercial experience tacky
3,@VirginAmerica it's really aggressive to blast...,0,really aggressive blast obnoxious entertainmen...
4,@VirginAmerica and it's a really big bad thing...,0,really big bad thing
5,@VirginAmerica seriously would pay $30 a fligh...,0,seriously would pay 30 flight seat playing rea...
6,"@VirginAmerica yes, nearly every time I fly VX...",1,yes nearly every time fly vx ear worm go away
...,...,...,...
14633,@AmericanAir my flight was Cancelled Flightled...,0,flight cancelled flightled leaving tomorrow mo...
14634,@AmericanAir right on cue with the delays👌,0,right cue delay
14635,@AmericanAir thank you we got on a different f...,1,thank got different flight chicago
14636,@AmericanAir leaving over 20 minutes Late Flig...,0,leaving 20 minute late flight warning communic...


In [16]:
#Dividindo os dados em treino e teste
X = df['clean_text']
y = df['airline_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Obs:** Agora, iremos vetorizar o texto aplicando as 3 técnicas propostas: Bag-of-Words, TF-IDF e Bag-of-nGrams

In [20]:
#Dividindo o treino e teste com Bag-of_words
vectorizer_bow = CountVectorizer()
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)


In [21]:
#Dividindo o treino e teste com TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)


In [22]:
#Dividindo o treino e teste com Bag-of-nGrams
vectorizer_ngrams = CountVectorizer(ngram_range=(1, 2))  
X_train_ngrams = vectorizer_ngrams.fit_transform(X_train)
X_test_ngrams = vectorizer_ngrams.transform(X_test)


Agora iremos aplicar um modelo de Regressão Logística em cada uma das técnicas abordadas acima

In [29]:
#Modelo com Bag of words
model_bow = LogisticRegression(max_iter = 1000)
model_bow.fit(X_train_bow, y_train)

#Realizando as predições
y_pred_bow = model_bow.predict(X_test_bow)

print("Avaliação do Modelo com Bag of Words")
print("Acurácia:", accuracy_score(y_test, y_pred_bow))
print("Precisão:", precision_score(y_test, y_pred_bow))
print("Recall:", recall_score(y_test, y_pred_bow))
print("F1-Score:", f1_score(y_test, y_pred_bow))

Avaliação do Modelo com Bag-of-Words
Acurácia: 0.9073342116820378
Precisão: 0.8075
Recall: 0.7067833698030634
F1-Score: 0.7537922987164527


In [30]:
#Modelo com TF-IDF
model_tfidf = LogisticRegression(max_iter=1000)
model_tfidf.fit(X_train_tfidf, y_train)

#Realizando as prdedições
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("\nAvaliação do Modelo com TF-IDF")
print("Acurácia:", accuracy_score(y_test, y_pred_tfidf))
print("Precisão:", precision_score(y_test, y_pred_tfidf))
print("Recall:", recall_score(y_test, y_pred_tfidf))
print("F1-Score:", f1_score(y_test, y_pred_tfidf))



Avaliação do Modelo com TF-IDF
Acurácia: 0.8959156785243741
Precisão: 0.8819444444444444
Recall: 0.5557986870897156
F1-Score: 0.6818791946308725


In [31]:
#Mmodelo com Bagof nGrams
model_ngrams = LogisticRegression(max_iter=1000)
model_ngrams.fit(X_train_ngrams, y_train)

#Realuzando as predições
y_pred_ngrams = model_ngrams.predict(X_test_ngrams)

print("\nAvaliação do Modelo com Bag of nGrams")
print("Acurácia:", accuracy_score(y_test, y_pred_ngrams))
print("Precisão:", precision_score(y_test, y_pred_ngrams))
print("Recall:", recall_score(y_test, y_pred_ngrams))
print("F1-Score:", f1_score(y_test, y_pred_ngrams))



Avaliação do Modelo com Bag of nGrams
Acurácia: 0.9073342116820378
Precisão: 0.8360655737704918
Recall: 0.6695842450765864
F1-Score: 0.7436208991494532


**Conclusões sobre o exercício 1:**